In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error, mean_absolute_percentage_error, explained_variance_score, max_error, accuracy_score 
from sklearn.linear_model import LinearRegression
import numpy as np 
import lightgbm as lgb 
import xgboost as xgb 
# from catboost import CatBoostRegressor # CatBoost убран
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Функция для расчета точности определения знака
def sign_accuracy(y_true, y_pred):
    return np.mean(np.sign(y_true) == np.sign(y_pred))

# Загрузка данных
# Убедитесь, что путь к файлу указан верно
try:
    lkoh_df = pd.read_csv('../data/features_final/LKOH_final.csv')
    print("Данные для LKOH загружены успешно.")
except FileNotFoundError:
    print("Файл LKOH_final.csv не найден. Проверьте путь к файлу.")
    # Можно добавить код для загрузки других тикеров или обработки ошибки

# Определение целевых переменных
targets = ['target_1d', 'target_3d', 'target_7d', 'target_30d', 'target_180d']

# Определение признаков оценки новостей для LKOH
# Для других тикеров 'LKOH' нужно будет заменить на соответствующий тикер
ticker_prefix = 'LKOH' # Это нужно будет менять для других тикеров

news_features = [
    f'{ticker_prefix}_news_score', f'{ticker_prefix}_news_score_roll_avg_5', 
    f'{ticker_prefix}_news_score_roll_avg_15', f'{ticker_prefix}_news_score_roll_avg_30',
    'WeightedIndices_news_score', 'WeightedIndices_news_score_roll_avg_5', 
    'WeightedIndices_news_score_roll_avg_15', 'WeightedIndices_news_score_roll_avg_30'
]

# Определение признаков оценки блогов для LKOH
blog_features = [
    f'{ticker_prefix}_blog_score', f'{ticker_prefix}_blog_score_roll_avg_5', 
    f'{ticker_prefix}_blog_score_roll_avg_15', f'{ticker_prefix}_blog_score_roll_avg_30',
    'WeightedIndices_blog_score', 'WeightedIndices_blog_score_roll_avg_5', 
    'WeightedIndices_blog_score_roll_avg_15', 'WeightedIndices_blog_score_roll_avg_30'
]

# Определение базовых признаков 
# (все, кроме целевых, новостных и блоговых, а также исключаем 'Date' и 'Ticker' и другие специфичные для таргетов колонки)
# Сначала получим все колонки
all_columns = lkoh_df.columns.tolist()

# Колонки, которые точно не являются признаками
excluded_columns = targets + news_features + blog_features + ['Date', 'Ticker']

# Дополнительно убедимся, что не включаем колонки, которые могут содержать информацию о будущем или являются вариациями таргетов
# Например, если есть колонки типа 'price_change_next_N_days' и т.п., их тоже надо исключить.
# В данном случае, предполагаем, что остальные колонки - это базовые признаки.
base_features = [col for col in all_columns if col not in excluded_columns]

# Удалим признаки, которые могут содержать NaN из-за rolling averages на начальных этапах, если они есть только в начале
# Это специфично для данных и требует их анализа. Пока оставим как есть.
# lkoh_df.dropna(subset=base_features + news_features + blog_features, inplace=True) # Раскомментировать и адаптировать при необходимости


print(f"Количество базовых признаков: {len(base_features)}")
if len(base_features) > 0:
    print(f"Пример базовых признаков: {base_features[:5]}...")
else:
    print("Базовые признаки не определены. Проверьте логику исключения колонок.")
    
print(f"Новостные признаки: {news_features}")
print(f"Блоговые признаки: {blog_features}")

# Проверка наличия всех признаков в DataFrame
missing_news_features = [f for f in news_features if f not in lkoh_df.columns]
missing_blog_features = [f for f in blog_features if f not in lkoh_df.columns]
missing_base_features = [f for f in base_features if f not in lkoh_df.columns] # Хотя это должно быть пусто по определению

if missing_news_features:
    print(f"ВНИМАНИЕ: Следующие новостные признаки отсутствуют в LKOH_final.csv: {missing_news_features}")
if missing_blog_features:
    print(f"ВНИМАНИЕ: Следующие блоговые признаки отсутствуют в LKOH_final.csv: {missing_blog_features}")

# Определим наборы признаков для экспериментов:
# 1. Только базовые признаки
features_set_1 = base_features
# 2. Базовые + новостные
features_set_2 = base_features + [f for f in news_features if f in lkoh_df.columns] # Используем только существующие
# 3. Базовые + новостные + блоговые
features_set_3 = base_features + [f for f in news_features if f in lkoh_df.columns] + [f for f in blog_features if f in lkoh_df.columns] # Используем только существующие

print(f"\\nКоличество признаков в наборе 1 (базовые): {len(features_set_1)}")
print(f"Количество признаков в наборе 2 (базовые + новости): {len(features_set_2)}")
print(f"Количество признаков в наборе 3 (базовые + новости + блоги): {len(features_set_3)}")

# Выведем первые несколько строк датафрейма для ознакомления
print("\\nПервые 5 строк DataFrame LKOH:")
print(lkoh_df.head())

# Проверим типы данных и наличие пропусков в целевых переменных
print("\\nИнформация о DataFrame LKOH:")
lkoh_df.info()

print("\\nПроверка на пропуски в целевых переменных LKOH:")
print(lkoh_df[targets].isnull().sum())

# Удаление строк с NaN в целевых переменных, если они есть
# lkoh_df.dropna(subset=targets, inplace=True)
# print("\\nРазмер DataFrame после удаления строк с NaN в таргетах:", lkoh_df.shape)

In [ ]:
# sign_accuracy должна быть определена глобально, обычно в первой ячейке.
# Убедитесь, что ячейка с ее определением выполнена.
# def sign_accuracy(y_true, y_pred):
#    return np.mean(np.sign(y_true) == np.sign(y_pred))

def train_and_evaluate_models(df, features, target_col, ticker_name, test_size=0.2, n_iter_search=10, cv_search=3):
    """
    Обучает и оценивает модели регрессии (LightGBM, XGBoost) 
    с подбором гиперпараметров через RandomizedSearchCV и расширенным набором метрик.
    """
    results = {}
    
    current_features = [f for f in features if f in df.columns]
    if not current_features:
        print(f"[{ticker_name} - {target_col}] Ни один из указанных признаков не найден. Пропуск.")
        return {}
    if target_col not in df.columns:
        print(f"[{ticker_name} - {target_col}] Целевая переменная '{target_col}' не найдена. Пропуск.")
        return {}

    X = df[current_features]
    y = df[target_col]

    combined_df = pd.concat([X, y], axis=1)
    combined_df.dropna(inplace=True)

    if combined_df.empty:
        print(f"[{ticker_name} - {target_col}] DataFrame пуст после удаления NaN. Пропуск.")
        return {}

    X = combined_df[current_features]
    y = combined_df[target_col]

    if X.empty or len(y) < (1/test_size) + cv_search: 
        print(f"[{ticker_name} - {target_col}] Недостаточно данных для обучения/тестирования/CV ({X.shape}, y: {y.shape}). Пропуск.")
        return {}

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)

    if X_train.empty or X_test.empty or len(X_train) < cv_search or len(y_train) < cv_search:
        print(f"[{ticker_name} - {target_col}] Тренировочная/тестовая выборка или данных для CV недостаточно. Пропуск.")
        return {}

    base_lgbm = lgb.LGBMRegressor(random_state=42, verbosity=-1)
    base_xgb = xgb.XGBRegressor(random_state=42, objective='reg:squarederror')

    param_dist_lgbm = {
        'n_estimators': [300, 500, 800],
        'learning_rate': [0.01, 0.05, 0.1],
        'num_leaves': [20, 31, 40, 50],
        'max_depth': [-1, 5, 10, 15],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
        'subsample': [0.7, 0.8, 0.9, 1.0]
    }

    param_dist_xgb = {
        'n_estimators': [300, 500, 800],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [3, 5, 7, 9],
        'subsample': [0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0],
        'gamma': [0, 0.1, 0.2]
    }

    models_config = {
        'LGBM': (base_lgbm, param_dist_lgbm),
        'XGBoost': (base_xgb, param_dist_xgb),
    }

    # Обновленный словарь scorers (удалена 'Accuracy')
    scorers = {
        'R2': r2_score,
        'MAE': mean_absolute_error,
        'MSE': mean_squared_error,
        'MedAE': median_absolute_error, 
        'MAPE': mean_absolute_percentage_error, 
        'ExplainedVariance': explained_variance_score, 
        'MaxError': max_error, 
        'Sign Accuracy': sign_accuracy 
        # 'Accuracy': accuracy_score # Удалено, т.к. это для классификации
    }

    for model_name, (model_base, params) in models_config.items():
        print(f"  Подбор параметров для {model_name}, таргет {target_col}...")
        
        current_cv = min(cv_search, len(X_train) // 2) 
        if len(X_train) < 2 or current_cv < 2: 
             print(f"    Недостаточно данных в X_train ({len(X_train)}) для кросс-валидации с {current_cv} фолдами. Пропуск {model_name}.")
             # Заполняем NaN для всех метрик, включая RMSE и best_params
             results[model_name] = {metric: np.nan for metric in scorers.keys()}
             results[model_name]['RMSE'] = np.nan
             results[model_name]['best_params'] = {}
             continue

        try:
            search = RandomizedSearchCV(
                estimator=model_base, 
                param_distributions=params, 
                n_iter=n_iter_search, 
                cv=current_cv, 
                scoring='r2', 
                random_state=42,
                n_jobs=-1 
            )
            search.fit(X_train, y_train)
            
            best_model = search.best_estimator_
            y_pred = best_model.predict(X_test)
            
            model_metrics = {}
            for metric_name, scorer_func in scorers.items():
                if metric_name == 'MAPE' and np.any(y_test == 0):
                    pass 
                model_metrics[metric_name] = scorer_func(y_test, y_pred)
            
            # Проверяем, есть ли MSE перед вычислением RMSE
            if 'MSE' in model_metrics and not np.isnan(model_metrics['MSE']):
                model_metrics['RMSE'] = np.sqrt(model_metrics['MSE'])
            else:
                model_metrics['RMSE'] = np.nan # Если MSE нет или NaN, RMSE тоже NaN
                
            model_metrics['best_params'] = search.best_params_
            results[model_name] = model_metrics
            print(f"    {model_name} (tuned) R2: {model_metrics.get('R2', float('nan')):.4f}, Sign Acc: {model_metrics.get('Sign Accuracy', float('nan')):.4f}")

        except Exception as e:
            print(f"Ошибка при подборе/обучении модели {model_name} для {target_col} тикера {ticker_name}: {e}")
            # Заполняем NaN для всех метрик, включая RMSE и best_params, в случае ошибки
            results[model_name] = {metric: np.nan for metric in scorers.keys()}
            results[model_name]['RMSE'] = np.nan
            results[model_name]['best_params'] = {}
            
    return results

print("Функция train_and_evaluate_models обновлена: удалена метрика 'Accuracy' и улучшена обработка RMSE.")

In [ ]:
import pandas as pd

# Читаем последние 10 строк из CSV файла
df = pd.read_csv('../data/processed/gpt/results_gpt_blogs.csv', engine='python').tail(10)

# Сохраняем копию в новый файл
output_path = '../data/processed/gpt/results_gpt_blogs_last_10.csv'
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"Копия последних 10 строк сохранена в {output_path}")
print("\nСодержимое файла:")
print(df)



In [ ]:
tickers_to_process = ['LKOH', 'SBER', 'GAZP'] # Добавьте сюда другие тикеры при необходимости
data_path_template = '../data/features_final/{}_final.csv'

all_experiment_results = []

# Глобально определенные целевые переменные (из предыдущей ячейки)
# targets = ['target_1d', 'target_3d', 'target_7d', 'target_30d', 'target_180d']

# Параметры для RandomizedSearchCV (можно настроить)
n_iterations_rscv = 30 # Уменьшено для скорости, можно увеличить для более тщательного поиска
cv_folds_rscv = 3      # Количество фолдов

for ticker in tickers_to_process:
    print(f"\nProcessing Ticker: {ticker} ============")
    file_path = data_path_template.format(ticker)
    
    try:
        df_ticker = pd.read_csv(file_path)
        print(f"Данные для {ticker} загружены успешно. Форма: {df_ticker.shape}")
    except FileNotFoundError:
        print(f"Файл {file_path} не найден. Пропуск тикера {ticker}.")
        continue
    except Exception as e:
        print(f"Ошибка при загрузке данных для {ticker}: {e}. Пропуск тикера.")
        continue

    # Динамическое определение новостных и блоговых признаков
    current_news_features = [
        f'{ticker}_news_score', f'{ticker}_news_score_roll_avg_5', 
        f'{ticker}_news_score_roll_avg_15', f'{ticker}_news_score_roll_avg_30',
        'WeightedIndices_news_score', 'WeightedIndices_news_score_roll_avg_5', 
        'WeightedIndices_news_score_roll_avg_15', 'WeightedIndices_news_score_roll_avg_30'
    ]
    current_blog_features = [
        f'{ticker}_blog_score', f'{ticker}_blog_score_roll_avg_5', 
        f'{ticker}_blog_score_roll_avg_15', f'{ticker}_blog_score_roll_avg_30',
        'WeightedIndices_blog_score', 'WeightedIndices_blog_score_roll_avg_5', 
        'WeightedIndices_blog_score_roll_avg_15', 'WeightedIndices_blog_score_roll_avg_30'
    ]

    # Отфильтруем только существующие в данном датасете признаки
    actual_news_features = [f for f in current_news_features if f in df_ticker.columns]
    actual_blog_features = [f for f in current_blog_features if f in df_ticker.columns]

    # Определение базовых признаков
    all_columns = df_ticker.columns.tolist()
    excluded_for_base = targets + actual_news_features + actual_blog_features
    # Добавим стандартные колонки, не являющиеся признаками (учитываем разные написания Date/date)
    potential_non_features = ['DATE', 'date', 'Ticker', 'SECID', 'TRADEDATE', 'tradetimestamp'] 
    for col_name in potential_non_features:
        if col_name in all_columns and col_name not in excluded_for_base:
            excluded_for_base.append(col_name)
    
    current_base_features = [col for col in all_columns if col not in excluded_for_base]
    
    if not current_base_features:
        print(f"Не найдено базовых признаков для {ticker} после исключения. Проверьте логику! Пропуск тикера.")
        continue
    print(f"Для {ticker}: Найдено {len(current_base_features)} базовых признаков.")

    # Определяем наборы признаков для экспериментов
    feature_sets_defs = {
        "BaseFeatures": current_base_features,
        "BasePlusNews": current_base_features + actual_news_features,
        "BasePlusNewsBlogs": current_base_features + actual_news_features + actual_blog_features
    }

    for target_col in targets:
        print(f"\n  Processing Target: {target_col} for Ticker: {ticker}")
        if target_col not in df_ticker.columns:
            print(f"    Целевая переменная {target_col} отсутствует в данных для {ticker}. Пропуск.")
            continue
            
        # Проверка на наличие достаточного количества не-NaN значений в таргете
        if df_ticker[target_col].isnull().all() or df_ticker[target_col].nunique() < 2:
            print(f"    Целевая переменная {target_col} для {ticker} содержит все NaN или только одно уникальное значение. Пропуск.")
            continue

        for fs_name, fs_features in feature_sets_defs.items():
            print(f"\n    Training with Feature Set: {fs_name} ({len(fs_features)} features)")
            
            # Убедимся, что в fs_features нет дубликатов (на случай если base уже содержал news/blog из-за ошибки в именовании)
            unique_fs_features = sorted(list(set(fs_features)))
            if len(unique_fs_features) == 0:
                print(f"      Набор признаков {fs_name} пуст для {ticker} - {target_col}. Пропуск.")
                continue

            # Создаем копию DataFrame для каждой итерации, чтобы избежать модификации оригинала при удалении NaNs
            # и для обработки NaNs специфично для текущего набора признаков и таргета
            df_loop_copy = df_ticker.copy()

            model_metrics_results = train_and_evaluate_models(
                df_loop_copy, 
                unique_fs_features, 
                target_col, 
                ticker,
                n_iter_search=n_iterations_rscv,
                cv_search=cv_folds_rscv
            )

            for model_name, metrics in model_metrics_results.items():
                result_row = {
                    'Ticker': ticker,
                    'Target': target_col,
                    'FeatureSet': fs_name,
                    'Model': model_name,
                }
                result_row.update(metrics) # Добавляем все метрики и best_params
                all_experiment_results.append(result_row)
                
                if metrics and 'R2' in metrics : # Проверяем что метрики не пустые
                    print(f"      {model_name} for {fs_name} -> R2: {metrics.get('R2', float('nan')):.4f}, SignAcc: {metrics.get('Sign Accuracy', float('nan')):.4f}")
                else:
                    print(f"      {model_name} for {fs_name} -> Обучение не удалось или нет метрик.")

# Преобразование результатов в DataFrame
results_df = pd.DataFrame(all_experiment_results)

print("\n==================================")
print("Эксперименты завершены!")
if not results_df.empty:
    print("Первые несколько строк DataFrame с результатами:")
    print(results_df.head())
    
    # Сохранение результатов в CSV (опционально)
    # results_df.to_csv('market_prediction_experiment_results.csv', index=False)
    # print("\nРезультаты сохранены в market_prediction_experiment_results.csv")
else:
    print("DataFrame с результатами пуст. Проверьте логи и возможные ошибки.")


In [ ]:
lkoh_df = pd.read_csv('../data/features_final/LKOH_final.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
lkoh_df.head()

In [ ]:
# Основной цикл исследования (с обновленными наборами признаков)

tickers_to_process = ['LKOH', 'SBER', 'GAZP']
data_path_template = '../data/features_final/{}_final.csv'

all_experiment_results_V2 = [] # Новая переменная для результатов с новым набором признаков

# Определяем целевые переменные (должны быть доступны из предыдущих ячеек)
# targets = ['target_1d', 'target_3d', 'target_7d', 'target_30d', 'target_180d']

# Параметры для RandomizedSearchCV
n_iterations_rscv = 50
cv_folds_rscv = 3      

# Новый, сокращенный список "самых важных" базовых признаков
selected_base_features_template = [
    'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 
    'RSI', 'MACD_Hist', 'Bear_Power', 'ATR', 'EMA_10', 'EMA_50', 'EMA_200', 'RTSI', 'EMV', 'ADX',
    'MOEXCN', 'MOEXIT', 'MOEXRE', 'MOEXEU', 'MOEXFN', 'MOEXINN', 'MOEXMM', 'MOEXOG', 'MOEXTL', 'MOEXTN', 'MOEXCH',
    'Revenue_y', 'NetProfit_y', 'ROE_y', 'Assets_q', 'NetProfit_q', 'PB_q',
    'BRENT_CLOSE', 'KEY_RATE', 'USD_RUB',
    'PE_y', 'PB_y'
]

for ticker in tickers_to_process:
    print(f"\nProcessing Ticker (V2 Features): {ticker} ============")
    file_path = data_path_template.format(ticker)
    
    try:
        df_ticker = pd.read_csv(file_path)
        print(f"Данные для {ticker} загружены успешно. Форма: {df_ticker.shape}")
    except FileNotFoundError:
        print(f"Файл {file_path} не найден. Пропуск тикера {ticker}.")
        continue
    except Exception as e:
        print(f"Ошибка при загрузке данных для {ticker}: {e}. Пропуск тикера.")
        continue

    # 1. Определение актуальных базовых признаков из нашего выбранного списка
    actual_selected_base_features = [f for f in selected_base_features_template if f in df_ticker.columns]
    if not actual_selected_base_features:
        print(f"Ни один из выбранных базовых признаков не найден для {ticker}. Пропуск тикера.")
        continue
    print(f"Для {ticker} (V2): Используется {len(actual_selected_base_features)} выбранных базовых признаков.")

    # 2. Определение новостных и блоговых признаков (только для тикера, без WeightedIndices)
    current_news_features_specific = [
        f'{ticker}_news_score', 
        f'{ticker}_news_score_roll_avg_5',
        f'{ticker}_news_score_roll_avg_30',
        'WeightedIndices_news_score'
    ]
    current_blog_features_specific = [
        f'{ticker}_blog_score',
        f'{ticker}_blog_score_roll_avg_5',
        f'{ticker}_blog_score_roll_avg_30',
        'WeightedIndices_blog_score'
    ]
    
    # Отфильтруем только существующие в данном датасете специфичные признаки
    actual_news_features_specific = [f for f in current_news_features_specific if f in df_ticker.columns]
    actual_blog_features_specific = [f for f in current_blog_features_specific if f in df_ticker.columns]

    # Определяем наборы признаков для экспериментов (V2)
    feature_sets_defs_V2 = {
        "BaseFeatures_V2": actual_selected_base_features,
        "BasePlusNews_V2": actual_selected_base_features + actual_news_features_specific,
        "BasePlusNewsBlogs_V2": actual_selected_base_features + actual_news_features_specific + actual_blog_features_specific
    }

    for target_col in targets:
        print(f"\n  Processing Target: {target_col} for Ticker: {ticker} (V2 Features)")
        if target_col not in df_ticker.columns:
            print(f"    Целевая переменная {target_col} отсутствует в данных для {ticker}. Пропуск.")
            continue
        if df_ticker[target_col].isnull().all() or df_ticker[target_col].nunique() < 2:
            print(f"    Целевая переменная {target_col} для {ticker} содержит все NaN или одно уникальное значение. Пропуск.")
            continue

        for fs_name, fs_features in feature_sets_defs_V2.items():
            print(f"\n    Training with Feature Set: {fs_name} ({len(fs_features)} features)")
            
            unique_fs_features = sorted(list(set(fs_features))) # Удаляем дубликаты, если вдруг появятся
            if not unique_fs_features:
                print(f"      Набор признаков {fs_name} пуст для {ticker} - {target_col}. Пропуск.")
                continue

            df_loop_copy = df_ticker.copy()
            model_metrics_results = train_and_evaluate_models(
                df_loop_copy, 
                unique_fs_features, 
                target_col, 
                ticker,
                n_iter_search=n_iterations_rscv,
                cv_search=cv_folds_rscv
            )

            for model_name, metrics in model_metrics_results.items():
                result_row = {
                    'Ticker': ticker,
                    'Target': target_col,
                    'FeatureSet': fs_name, # Используем новые имена наборов признаков
                    'Model': model_name,
                }
                result_row.update(metrics)
                all_experiment_results_V2.append(result_row)
                
                if metrics and 'R2' in metrics :
                    print(f"      {model_name} for {fs_name} -> R2: {metrics.get('R2', float('nan')):.4f}, SignAcc: {metrics.get('Sign Accuracy', float('nan')):.4f}")
                else:
                    print(f"      {model_name} for {fs_name} -> Обучение не удалось или нет метрик.")

# Преобразование результатов (V2) в DataFrame
results_df_V2 = pd.DataFrame(all_experiment_results_V2)

print("\n==================================")
print("Эксперименты с V2 наборами признаков завершены!")
if not results_df_V2.empty:
    print("Первые несколько строк DataFrame с результатами (V2 Features):")
    print(results_df_V2.head())
    
    # Сохранение результатов в CSV (опционально)
    # results_df_V2.to_csv('market_prediction_experiment_results_V2.csv', index=False)
    # print("\nРезультаты (V2) сохранены в market_prediction_experiment_results_V2.csv")
else:
    print("DataFrame с результатами (V2 Features) пуст. Проверьте логи и возможные ошибки.")